<h4><a href="https://page.mi.fu-berlin.de/scravy/realworldhaskell/materialien/monad-transformers-step-by-step.pdf">From this paper</a></h4>

In [1]:
import Control.Monad.Identity
import Control.Monad.Error
import Control.Monad.Reader
import Control.Monad.State
import Control.Monad.Writer
import Data.Maybe
import qualified Data.Map as Map

In [2]:
type Name = String

data Exp = Lit Integer
    | Var Name
    | Plus Exp Exp
    | Abs Name Exp
    | App Exp Exp
    deriving Show

In [3]:
type Env = Map.Map Name Value

data Value = IntVal Integer
    | FunVal Env Name Exp
    deriving Show

In [4]:
eval0 :: Env -> Exp -> Value
eval0 _ (Lit i) = IntVal i
eval0 env (Var v) = fromJust (Map.lookup v env)
eval0 env (Plus exp1 exp2) = let IntVal i1 = eval0 env exp1
                                 IntVal i2 = eval0 env exp2
                             in IntVal(i1 + i2)
eval0 env (Abs n e) = FunVal env n e
eval0 env (App e1 e2) = let val1 = eval0 env e1
                            val2 = eval0 env e2
                        in case val1 of
                            FunVal env' n body -> eval0 (Map.insert n val2 env') body

In [5]:
-- 12+((λx → x)(4+2))
exampleEx = Lit 12 `Plus` App (Abs "x" (Var "x")) (Lit 4 `Plus` Lit 2)

In [6]:
eval0 Map.empty exampleEx

IntVal 18

<h4>Converting to Monadic Style</h4>

In [25]:
liftReturn :: Monad m => Maybe a -> m a
liftReturn Nothing = fail "Key not found in map"
liftReturn (Just a) = return a

In [26]:
type Eval1 a = Identity a

runEval1 :: Eval1 a -> a
runEval1 = runIdentity

In [31]:
eval1 :: Env -> Exp -> Eval1 Value
eval1 _ (Lit i) = return (IntVal i)
eval1 env (Var v) = liftReturn $ Map.lookup v env
eval1 env (Plus exp1 exp2) = do
    IntVal i1 <- eval1 env exp1
    IntVal i2 <- eval1 env exp2
    return $ IntVal (i1 + i2)
eval1 env (Abs n e) = return (FunVal env n e)
eval1 env (App e1 e2) = do
    val1 <- eval1 env e1
    val2 <- eval1 env e2
    case val1 of
        FunVal env' n body -> eval1 (Map.insert n val2 env') body

In [32]:
runEval1 (eval1 Map.empty exampleEx)

IntVal 18

<h4>Adding Error Handling</h4>

In [33]:
type Eval2 a = ErrorT String Identity a

runEval2 :: Eval2 a -> Either String a
runEval2 ev = runIdentity (runErrorT ev)

In [34]:
eval2a :: Env -> Exp -> Eval2 Value
eval2a _ (Lit i) = return (IntVal i)
eval2a env (Var v) = liftReturn $ Map.lookup v env
eval2a env (Plus exp1 exp2) = do
    IntVal i1 <- eval2a env exp1
    IntVal i2 <- eval2a env exp2
    return $ IntVal (i1 + i2)
eval2a env (Abs n e) = return (FunVal env n e)
eval2a env (App e1 e2) = do
    val1 <- eval2a env e1
    val2 <- eval2a env e2
    case val1 of
        FunVal env' n body -> eval2a (Map.insert n val2 env') body

In [35]:
runEval2 (eval2a Map.empty exampleEx)

Right (IntVal 18)

In [36]:
eval2b :: Env -> Exp -> Eval2 Value
eval2b _ (Lit i) = return (IntVal i)
eval2b env (Var v) = liftReturn $ Map.lookup v env
eval2b env (Plus exp1 exp2) = do
    e1' <- eval2b env exp1
    e2' <- eval2b env exp2
    case (e1', e2') of
        (IntVal i1, IntVal i2) -> return $ IntVal (i1 + i2)
        _ -> throwError "type Error"
eval2b env (Abs n e) = return (FunVal env n e)
eval2b env (App e1 e2) = do
    val1 <- eval2b env e1
    val2 <- eval2b env e2
    case val1 of
        FunVal env' n body -> eval2b (Map.insert n val2 env') body
        _                  -> throwError "type Error"

In [37]:
runEval2 (eval2a Map.empty (Plus (Lit 1) (Abs "x" (Var "x"))))

Left "Pattern match failure in do expression at <interactive>:6:5-13"

In [38]:
runEval2 (eval2b Map.empty (Var "x"))

Left "Key not found in map"

In [39]:
eval2c :: Env -> Exp -> Eval2 Value
eval2c _ (Lit i) = return (IntVal i)
eval2c env (Var v) = liftReturn $ Map.lookup v env
eval2c env (Plus exp1 exp2) = do
    IntVal i1 <- eval2c env exp1
    IntVal i2 <- eval2c env exp2
    return $ IntVal (i1 + i2)
eval2c env (Abs n e) = return (FunVal env n e)
eval2c env (App e1 e2) = do
    FunVal env' n body <- eval2c env e1
    val2               <- eval2c env e2
    eval2c (Map.insert n val2 env') body

<h4>Final version</h4>

In [40]:
eval2 :: Env -> Exp -> Eval2 Value
eval2 _ (Lit i) = return (IntVal i)
eval2 env (Var v) = case Map.lookup v env of
    Nothing -> throwError ("unbound variable: " ++ v)
    Just val -> return val
eval2 env (Plus exp1 exp2) = do
    e1' <- eval2 env exp1
    e2' <- eval2 env exp2
    case (e1', e2') of
        (IntVal i1, IntVal i2) -> return $ IntVal (i1 + i2)
        _ -> throwError "type Error in addition"
eval2 env (Abs n e) = return (FunVal env n e)
eval2 env (App e1 e2) = do
    val1 <- eval2 env e1
    val2 <- eval2 env e2
    case val1 of
        FunVal env' n body -> eval2 (Map.insert n val2 env') body
        _                  -> throwError "type error in application"

<h4>Hide the env using ReaderT</h4>

In [44]:
type Eval3 a = ReaderT Env (ErrorT String Identity) a

runEval3 :: Env -> Eval3 a -> Either String a
runEval3 env ev = runIdentity (runErrorT (runReaderT ev env))

In [47]:
eval3 :: Exp -> Eval3 Value
eval3 (Lit i) = return (IntVal i)
eval3 (Var n) = do
    env <- ask
    case Map.lookup n env of
        Nothing  -> throwError ("unbound variable: " ++ n)
        Just val -> return val
eval3 (Plus e1 e2) = do
    e1' <- eval3 e1
    e2' <- eval3 e2
    case (e1', e2') of
        (IntVal i1, IntVal i2) -> return (IntVal (i1 + i2))
        _                      -> throwError "type error in addition"
eval3 (Abs n e) = do
    env <- ask
    return (FunVal env n e)
eval3 (App e1 e2) = do
    val1 <- eval3 e1
    val2 <- eval3 e2
    case val1 of
        FunVal env' n body -> local (const (Map.insert n val2 env')) (eval3 body)
        _                  -> throwError "type error in application"

In [48]:
runEval3 Map.empty (eval3 exampleEx)

Right (IntVal 18)

<h4>Adding State</h4>

In [49]:
type Eval4 a = ReaderT Env (ErrorT String (StateT Integer Identity)) a

runEval4 :: Env -> Integer -> Eval4 a -> (Either String a, Integer)
runEval4 env st ev = runIdentity (runStateT (runErrorT (runReaderT ev env)) st)

In [50]:
-- increment the count of number of evaluation steps
tick :: (Num s, MonadState s m) => m ()
tick = do
    st <- get
    put (st+1)

In [51]:
eval4 :: Exp -> Eval4 Value
eval4 (Lit i) = do
    tick
    return (IntVal i)
eval4 (Var n) = do
    tick
    env <- ask
    case Map.lookup n env of
        Nothing  -> throwError ("unbound variable: " ++ n)
        Just val -> return val
eval4 (Plus e1 e2) = do
    tick
    e1' <- eval4 e1
    e2' <- eval4 e2
    case (e1', e2') of
        (IntVal i1, IntVal i2) -> return (IntVal (i1+i2))
        _                      -> throwError "type error in addition"
eval4 (Abs n e) = do
    tick
    env <- ask
    return (FunVal env n e)
eval4 (App e1 e2) = do
    tick
    val1 <- eval4 e1
    val2 <- eval4 e2
    case val1 of
        FunVal env' n body -> local (const (Map.insert n val2 env')) (eval4 body)
        _                  -> throwError "type error in application"

In [54]:
runEval4 Map.empty 0 (eval4 exampleEx)

(Right (IntVal 18),8)

<h4>Adding logging (using Writer)</h4>

In [55]:
type Eval5 a = ReaderT Env (ErrorT String (WriterT [String] (StateT Integer Identity))) a

runEval5 :: Env -> Integer -> Eval5 a -> ((Either String a, [String]), Integer)
runEval5 env st ev = runIdentity (runStateT (runWriterT (runErrorT (runReaderT ev env))) st)

In [57]:
eval5 :: Exp -> Eval5 Value
eval5 (Lit i) = do
    tick
    return (IntVal i)
eval5 (Var n) = do
    tick
    tell [n]
    env <- ask
    case Map.lookup n env of
        Nothing  -> throwError ("unbound variable: " ++ n)
        Just val -> return val
eval5 (Plus e1 e2) = do
    tick
    e1' <- eval5 e1
    e2' <- eval5 e2
    case (e1', e2') of
        (IntVal i1, IntVal i2) -> return (IntVal (i1+i2))
        _                      -> throwError "type error in addition"
eval5 (Abs n e) = do
    tick
    env <- ask
    return (FunVal env n e)
eval5 (App e1 e2) = do
    tick
    val1 <- eval5 e1
    val2 <- eval5 e2
    case val1 of
        FunVal env' n body -> local (const (Map.insert n val2 env')) (eval5 body)
        _                  -> throwError "type error in application"

In [58]:
runEval5 Map.empty 0 (eval5 exampleEx)

((Right (IntVal 18),["x"]),8)

<h4>With IO</h4>

In [60]:
type Eval6 a = ReaderT Env (ErrorT String (WriterT [String] (StateT Integer IO))) a

runEval6 :: Env -> Integer -> Eval6 a -> IO ((Either String a, [String]), Integer)
runEval6 env st ev = runStateT (runWriterT (runErrorT (runReaderT ev env))) st

In [63]:
eval6 :: Exp -> Eval6 Value
eval6 (Lit i) = do
    tick
    liftIO $ print i
    return $ IntVal i
eval6 (Var n) = do
    tick
    tell [n]
    env <- ask
    case Map.lookup n env of
        Nothing -> throwError ("unbound variable: " ++ n)
        Just val -> return val
eval6 (Plus e1 e2) = do
    tick
    e1' <- eval6 e1
    e2' <- eval6 e2
    case (e1', e2') of
        (IntVal i1, IntVal i2) -> return (IntVal (i1+i2))
        _                      -> throwError "type error in addition"
eval6 (Abs n e) = do
    tick
    env <- ask
    return (FunVal env n e)
eval6 (App e1 e2) = do
    tick
    val1 <- eval6 e1
    val2 <- eval6 e2
    case val1 of
        FunVal env' n body -> local (const (Map.insert n val2 env')) (eval6 body)
        _                  -> throwError "type error in application"

In [64]:
runEval6 Map.empty 0 (eval6 exampleEx)

12
4
2
((Right (IntVal 18),["x"]),8)